### 1 Non-Clifford + Projection onto state $\ket{1}$
- Consider circuit $U$ with one non-Clifford gate ($T$ rotation gate) and a projection onto the one state on only one qubit ($\ket{1}\bra{1}$)
- Overarching Goal: Compute largest eigenvalue of $\bra{0^m} U^{\dagger} \ket{1}\bra{1} U \ket{0^m} \implies$ finding smallest eigenvalue of $\bra{0^m} U^{\dagger} Z U \ket{0^m}$



In [2]:
import stim
import numpy as np

### Adding T-gate Conjugation Functionality
* One of the drawbacks of using stim is that it exclusively makes use of stabilizer circuits
* Therefore, need to come up with a way to incorporate the behavior of conjugating Paulis by a T-gate
* Options:
    * Define a number of shots for random cliffords. Iterating over this number of shots, randomly, instead of placing a clifford gate, simulate action of T-gate on 
    current Pauli (I believe this will be easier to implement)
        * Define number of shots to sample cliffords
        * Start with pre-determined number of $T$-gates (Start with 1 for now)
        * For one of the shots (choose random integer from 0 to num_shots - 1) apply T gate as opposed to random clifford
        * Have list of tableaus where each element of list will be a summand.
        * Apply current stabilizer tableau to each element of list
        * In the end, there should be a list of tableaus on which we can apply each to a z-gate on the first qubit (via conjugation) to get out Sum of Paulis
    * Try to download stim locally and define a T-gate so that it is symbolically represented as a sum of stabilizer tableaus.
        * It can be written in the following way:
        $$T = \bigg(\cos \bigg(\frac{\pi}{8}\bigg) - \sin \bigg(\frac{\pi}{8}\bigg)\bigg) \mathbb{I} + e^{-i\pi/4} \sqrt{2}sin \bigg(\frac{\pi}{8}\bigg) S$$

In [17]:
num_wit_qubits = 4 # variable 'n' in paper
# For now, what the witness qubits are initialized to be doesn't matter (I THINK)
# as we are trying to minimize Val, given some optimal input states.
wit_qubits = np.zeros(num_wit_qubits, dtype=complex)

num_anc_qubits = 0 # variable 'm' in paper
anc_qubits = np.zeros(num_anc_qubits, dtype=complex)

num_t_gate = 1 # variable 't' in paper

num_meas_qubits = 1 # variable 'k' in paper
total_qubits = num_wit_qubits + num_anc_qubits

In [18]:
t = stim.Tableau(total_qubits) # Initialize identity tableau
num_shots = 15 # Number of layers of gates we would like to apply
t_gate_loc = [] # List of shot numbers corresponding to T-gate application
t_gate_loc_count = 0;
tableau_list = [] # List of tableaus, where each element will correspond to summands in sum of Paulis of result

pauli_z_string = "Z" + "_" * (total_qubits - 1)
z = stim.PauliString(pauli_z_string)
pauli_sum = [[1, z]]
#pauli_sum = np.array(pauli_sum)

# Create list of shot numbers where we would like to apply T-gate
for _ in range(num_t_gate):
    num_to_add = np.random.randint(0, num_shots)
    if (num_to_add in t_gate_loc):
        continue
    else:
        t_gate_loc.append(num_to_add)

print(t_gate_loc)

# Need to sort this array so that application of T-gates makes chronological sense with elements of 't_gate_loc'
t_gate_loc = np.sort(t_gate_loc) 

# Update tableau over number of shots
#   - If have to apply T-gate:
#       *  
for i in range(num_shots):
    if (t_gate_loc_count < len(t_gate_loc) and i == t_gate_loc[t_gate_loc_count]):
        # Apply tableau so far on current Pauli in center
        # (Since there is only one T-gate, we start with just Pauli Z on the first qubit)
        interim_conj = t(pauli_sum[0][1])

        # Update the 'pauli_sum' array to reflect application of tableau
        pauli_sum[0][0] = pauli_sum[0][0] * interim_conj.sign
        pauli_sum[0][1] = interim_conj

        # Check Pauli on first qubit (hence the index we check is 1)
        if (interim_conj.__str__()[1] == 'I' or interim_conj.__str__()[1] == 'Z'):
            continue
        elif (interim_conj.__str__()[1] == 'X'):
            new_summand = [pauli_sum[0][0] * np.sqrt(2)/2, stim.PauliString(interim_conj.__str__()[0] + 'Y' + interim_conj.__str__()[2:])]
            pauli_sum[0][0] = new_summand[0]
            pauli_sum.append(new_summand)
        else:
            new_summand = [-1 * pauli_sum[0][0] * np.sqrt(2)/2, stim.PauliString(interim_conj.__str__()[0] + 'Y' + interim_conj.__str__()[2:])]
            pauli_sum[0][0] = new_summand[0] * -1
            pauli_sum.append(new_summand)

            t_gate_loc_count += 1
            t = stim.Tableau(total_qubits)
    else:
        t.append(stim.Tableau.random(total_qubits), list(range(total_qubits)))

for elem in pauli_sum:
    elem[1] = t(elem[1])
    elem[0] = elem[0] * elem[1].sign

print(pauli_sum)

[6]
[[(0.7071067811865476-0j), stim.PauliString("-YX")], [(-0.7071067811865476+0j), stim.PauliString("-YX")]]
